<table>
<tr>
<td width=15%><img src="./img/UGA.png"></img></td>
<td><center><h1>Introduction à la Recherche Opérationelle</h1><br/>M2 Statistique Science des Données (SSD)</center></td>
<td width=15%>
<a href="https://www-ljk.imag.fr/membres/Anatoli.Iouditski/" style="font-size: 16px; font-weight: bold">Anatoli Juditsky</a><br/>
<a href="http://www.iutzeler.org" style="font-size: 16px; font-weight: bold">Franck Iutzeler</a><br/>
Version Py2019 </td>
</tr>
</table>

<br/><br/>
<center><a style="font-size: 30pt; font-weight: bold">TP 1 - Planification de Regime </a></center>
<br/><br/>

# 1. Récupération des données

### a. Menu

In [1]:
import cvxpy as cp
import pandas as pd
import numpy as np

In [2]:
foods = pd.read_csv("data/food.dat",sep='\t')
n,p = foods.shape 

In [3]:
foods

,Food,Cal,CalFat,Fat,SatFat,Chol,Sodium,Carbo,Protein,VitA,VitC,Calcium,Iron
0,1%_Low_Fat_Milk_Jug 1_carton_(236_ml) ...,100,20,2,1,10,125,12,8,10,4,30,0
1,Apple_Slices 1.2_oz_(34_g) ...,15,0,0,0,0,0,4,0,0,160,2,0
2,BBQ_Ranch_Burger 4.1_oz_(116_g) ...,350,140,16,6,45,680,37,16,4,0,20,15
3,"Bacon,_Egg_&_Cheese_Bagel 7_oz_(199_g) ...",630,290,32,11,275,1490,57,30,20,15,20,20
4,"Bacon,_Egg_&_Cheese_Bagel_with_Egg_Whites 7.2_...",580,230,26,9,60,1490,55,30,10,15,20,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,Vanilla_McCafe_Shake_(12_fl_oz_cup) 12_fl_oz ...,530,140,15,10,60,160,86,11,20,0,40,0
301,Vanilla_McCafe_Shake_(16_fl_oz_cup) 16_fl_oz ...,660,170,19,12,75,200,109,14,25,0,50,0
302,Vanilla_McCafe_Shake_(22_fl_oz_cup) 22_fl_oz ...,820,210,23,15,90,260,135,18,30,0,60,0
303,Vanilla_Reduced_Fat_Ice_Cream_Cone 3.7_oz_(105...,170,40,4,3,15,70,27,5,6,0,15,2


In [4]:
n

305

# 2. Alimentation idéale

In [5]:
nutr = pd.read_csv("data/nutr_ideal.dat",sep=' ')
nutr

,Cal,CalFat,Fat,SatFat,Chol,Sodium,Carbo,Protein,VitA,VitC,Calcium,Iron
0,2500,600,65,20,300,2400,300,50,100,100,100,100


### a. Combinaisons pour une alimentation pauvre en calories

Pour commencer, nous allons essayer de trouver une stratégie $x$ en résolvant un problème `diet1` de minimisation des calories, sous contrainte de rester idéal vis à vis des autres nutriments (CalFat, Fat, VitA, etc.)

> **Question:** Implementer les fonctions et contraintes suivantes, puis résoudre le problème.

* Fonction objective linéaire à minimiser: $c^\mathrm{T}x$ où $c$ contient les calories par aliment

In [6]:
x = cp.Variable(n) 

cal = np.array(foods["Cal"]) # calories

diet1 = cp.Minimize(cal*x)

* Contrainte à vérifier: $x$ est positif (par de quantités négatives d'aliments) et $Ax=b$ où $A$ contient les valeurs des nutriments par aliment et $b$ contient les valeurs idéales (sans les calories)

In [7]:
# Constraints matrix
A = np.array(foods.drop(columns=["Food","Cal"])).T
b = np.array(nutr)[0,1:12]

In [8]:
A

array([[ 20,   0, 140, ..., 210,  40,  40],
       [  2,   0,  16, ...,  23,   4,   4],
       [  1,   0,   6, ...,  15,   3,   1],
       ...,
       [  4, 160,   0, ...,   0,   0,   0],
       [ 30,   2,  20, ...,  60,  15,   0],
       [  0,   0,  15, ...,   0,   2,   0]])

In [9]:
b

array([ 600,   65,   20,  300, 2400,  300,   50,  100,  100,  100,  100])

In [10]:
contrainte1 = x>= 0.0
contrainte2 = A*x - b == 0.0

* Resolution par CVXR

Attention, utilisez le Solver "ECOS" 

        prob.solve(verbose=True,solver="ECOS")
        
le solveur par défaut peut donner de fausses informations...

In [11]:
prob =  cp.Problem(diet1 , constraints = [contrainte1, contrainte2])
prob.solve(verbose=True,solver="ECOS")


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +2.013e+03  +2.013e+03  +3e+03  7e-01  8e-02  1e+00  1e+01    ---    ---    1  1  - |  -  - 
 1  +1.933e+03  +1.938e+03  +2e+03  1e+00  1e-01  7e+00  8e+00  0.4808  4e-01   1  1  1 |  0  0
 2  +1.177e+03  +1.192e+03  +4e+02  6e-01  8e-02  2e+01  1e+00  0.8448  3e-02   1  1  1 |  0  0
 3  +1.190e+03  +1.195e+03  +5e+01  2e-01  1e-02  6e+00  2e-01  0.8924  1e-02   1  1  1 |  0  0
 4  +1.607e+03  +1.621e+03  +2e+01  5e-01  1e-02  1e+01  7e-02  0.7035  2e-01   1  1  1 |  0  0
 5  +1.983e+03  +2.054e+03  +3e+00  5e-01  1e-02  7e+01  1e-02  0.8444  6e-03   1  1  1 |  0  0
 6  +1.941e+03  +7.645e+03  +4e-02  5e-01  1e-02  6e+03  1e-04  0.9890  2e-03   1  1  1 |  0  0
 7  +1.941e+03  +5.161e+05  +5e-04  5e-01  1e-02  5e+05  2e-06  0.9890  1e-04   1  1  1 |  0  0
 8  +1.941e+03  +4.636e+07  +5e-06  5e-01  1e-

inf

In [12]:
prob.status

'infeasible'

### b. Relaxation des contraintes


Le problème précedent n'avait pas de point faisable avec les contraintes imposées. Afin d'obtenir des points faisable, nous allons remplacer la contrainte
$$ Ax = b$$ 
par les deux contraintes de type *boite*
$$ Ax \geq 0.8 b ~~ \text{ et  } ~~ Ax \leq b $$
ainsi les contraintes de nutriments sont satisfaites entre 80 et 100% et le problème devient faisable.

> **Question:** Définir les deux contraintes de boite et resoudre le problème. Vérifier qu'il est faisable.

In [13]:
contrainte2a = A*x <= 1.0*b 
contrainte2b = A*x >= 0.8*b

In [14]:
prob2= cp.Problem(diet1 , constraints = [contrainte1, contrainte2a,contrainte2b])
prob2.solve(verbose=True,solver="ECOS")


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +1.808e+03  +8.185e+02  +2e+04  2e-01  3e-01  1e+00  5e+01    ---    ---    1  1  - |  -  - 
 1  +1.736e+03  +1.219e+03  +8e+03  1e-01  1e-01  7e+00  3e+01  0.5884  2e-01   1  1  1 |  0  0
 2  +1.596e+03  +1.437e+03  +3e+03  3e-02  4e-02  6e+00  1e+01  0.8010  2e-01   1  1  1 |  0  0
 3  +1.293e+03  +1.005e+03  +3e+03  6e-02  8e-02  2e+01  9e+00  0.2964  6e-01   1  1  1 |  0  0
 4  +1.243e+03  +1.100e+03  +1e+03  3e-02  4e-02  1e+01  4e+00  0.5997  1e-01   1  1  1 |  0  0
 5  +1.329e+03  +1.216e+03  +1e+03  2e-02  3e-02  8e+00  4e+00  0.4499  7e-01   1  1  1 |  0  0
 6  +1.373e+03  +1.332e+03  +4e+02  9e-03  1e-02  3e+00  1e+00  0.6741  4e-03   1  1  1 |  0  0
 7  +1.398e+03  +1.378e+03  +2e+02  5e-03  5e-03  2e+00  6e-01  0.7476  3e-01   1  1  1 |  0  0
 8  +1.397e+03  +1.390e+03  +7e+01  2e-03  2e-

1397.0487147820581

In [15]:
prob2.status

'optimal'

In [16]:
x.value

array([ 6.16117845e-12,  2.36008897e-01,  1.23342457e-11,  2.61398894e-12,
        1.19737829e-12,  8.57558589e-12,  5.07316521e-12,  3.34377195e-12,
        2.22466103e-12,  5.39549310e-12,  2.36453544e-12,  6.80724195e-12,
        4.09753446e-12,  6.24063698e-12,  3.21615511e-12,  9.23998511e-12,
        5.61732485e-12,  9.78024891e-13,  1.13548106e-12,  2.84695130e-12,
        3.58490781e-12,  7.66824862e-13,  9.44620369e-13,  1.23443315e-11,
       -3.08042357e-12, -2.44788719e-12, -1.97079899e-12,  1.68292412e-11,
       -2.78972580e-12, -3.28601264e-12, -2.76110842e-12, -2.28510982e-12,
       -2.39066464e-12, -2.32575630e-12,  2.23397346e-11,  8.52785222e-13,
       -3.73604475e-13, -5.60964726e-13,  4.42255061e-12,  3.07328765e-12,
       -3.17290065e-13,  5.12209773e-12,  7.67096922e-12,  3.69308368e+00,
       -2.16115030e-12, -2.26698406e-12, -2.09346141e-12,  2.86086311e-12,
        9.88093255e-13, -2.88492900e-12, -2.27386436e-12, -1.01962011e-12,
        3.20976010e+00,  

> **Question:** Affichez les valeur des nutriments pour la solution obtenue et la comparer à 80% et 100% des valeurs prescrites.

In [17]:
base = nutr.drop(columns=["Cal"]).T
base.columns = ["1*b"]

In [18]:
base["computed"] = np.dot(A,x.value)

In [19]:
base["0.8*b"] = 0.8*b

In [20]:
base

,1*b,computed,0.8*b
CalFat,600,480.000000,480.0
Fat,65,54.081051,52.0
SatFat,20,20.000000,16.0
Chol,300,240.000000,240.0
Sodium,2400,2200.662407,1920.0
Carbo,300,244.665407,240.0
Protein,50,50.000000,40.0
VitA,100,100.000000,80.0
VitC,100,80.000000,80.0
Calcium,100,80.000000,80.0


> **Question:** Donner le nombre de calories totales et les aliments pour lesquelles la quantité est supérieure à 0.3.

In [21]:
xopt = x.value

In [22]:
np.dot(cal ,xopt)

1397.0487147820577

In [23]:
for j in range(n):
    if xopt[j]>0.3:
        print("{} \t ({} cal) \t : \t {}".format(foods.iloc[j,0],foods.iloc[j,1],xopt[j]))


Chocolate_Chip_Cookie 1_cookie_(33_g)                      	 (160 cal) 	 : 	 3.6930836797788604
Coffee_(Large) 16_fl_oz_cup                                	 (0 cal) 	 : 	 3.2097600967356694
Coffee_(Medium) 16_fl_oz_cup                               	 (0 cal) 	 : 	 3.2097600967356694
Coffee_(Small) 12_fl_oz_cup                                	 (0 cal) 	 : 	 3.2097600967356694
Dasani_Water 16.9_fl_oz                                    	 (0 cal) 	 : 	 3.2097600967356694
Diet_Coke_(Child) 12_fl_oz_cup                             	 (0 cal) 	 : 	 1.5834853369668858
Diet_Coke_(Large) 30_fl_oz_cup                             	 (0 cal) 	 : 	 1.2445618254171356
Diet_Coke_(Medium) 21_fl_oz_cup                            	 (0 cal) 	 : 	 1.4621342658462242
Diet_Coke_(Small) 16_fl_oz_cup                             	 (0 cal) 	 : 	 1.7670220855252274
EQUAL_0_Calorie_Sweetener 1_pkg_(1.0_g)                    	 (0 cal) 	 : 	 30.936033669720565
Egg_McMuffin 4.8_oz_(135_g)                               

On remarque que la solution n'est pas pratique! 

### c. D'autres contraintes

On va imposer:
* pas de `EQUAL_0_Calorie_Sweetener 1_pkg_(1.0_g)` (item $73$) ni de `SPLENDA_No_Calorie_Sweetener 1_pkg_(1.0_g) :  30.9360336697206` (item $248$)  car c'est pas très bon
* pas plus de 10 unités dans le menu, pas plus de 2 unités par item
* au moins un petit déjeuner (items 3 à 22)

Relacher au besoin les contraintes de nutrition...

> **Question :** Implémenter ces contraintes une par une et regarder la valeur optimale de la fonction (le nombre de calories). Cette valeur augmente-t-elle ou diminue-t-elle ? Pourquoi ?

In [24]:
contrainte3a = x[73] == 0
contrainte3b = x[248] == 0

In [25]:
prob3= cp.Problem(diet1 , constraints = [contrainte1, contrainte2a,contrainte2b, contrainte3a, contrainte3b])
prob3.solve(verbose=True,solver="ECOS")


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +1.808e+03  +8.182e+02  +2e+04  2e-01  3e-01  1e+00  5e+01    ---    ---    1  1  - |  -  - 
 1  +1.746e+03  +1.231e+03  +8e+03  1e-01  1e-01  7e+00  3e+01  0.5836  2e-01   1  1  1 |  0  0
 2  +1.620e+03  +1.463e+03  +3e+03  3e-02  4e-02  5e+00  1e+01  0.7975  3e-01   1  1  1 |  0  0
 3  +1.605e+03  +1.557e+03  +1e+03  1e-02  1e-02  3e+00  4e+00  0.8698  3e-01   1  1  1 |  0  0
 4  +1.595e+03  +1.578e+03  +4e+02  3e-03  4e-03  8e-01  1e+00  0.8209  2e-01   1  1  1 |  0  0
 5  +1.589e+03  +1.583e+03  +2e+02  1e-03  2e-03  3e-01  5e-01  0.6595  8e-02   1  1  1 |  0  0
 6  +1.586e+03  +1.584e+03  +7e+01  5e-04  7e-04  1e-01  2e-01  0.6479  1e-01   1  1  1 |  0  0
 7  +1.585e+03  +1.585e+03  +3e+01  2e-04  3e-04  4e-02  8e-02  0.7791  2e-01   1  1  1 |  0  0
 8  +1.585e+03  +1.585e+03  +2e+00  2e-05  2e-

1584.5315377055053

In [26]:
prob2.value

1397.0487147820581

In [27]:
x.value[73]

1.0655888912818066e-13

In [28]:
foods.iloc[73,0]

'EQUAL_0_Calorie_Sweetener 1_pkg_(1.0_g)                   '

Contrainte suivante

In [29]:
contrainte4a = cp.sum(x) <= 10
contrainte4b = x <= 2.0

In [30]:
prob4= cp.Problem(diet1 , constraints = [contrainte1, contrainte2a,contrainte2b, contrainte3a, contrainte3b, contrainte4a, contrainte4b])
prob4.solve(verbose=True,solver="ECOS")


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +1.832e+03  -8.810e+04  +1e+05  2e-01  7e-02  1e+00  2e+02    ---    ---    1  1  - |  -  - 
 1  +1.647e+03  -1.282e+04  +3e+04  3e-02  1e-02  2e+01  4e+01  0.8943  1e-01   1  1  1 |  0  0
 2  +1.648e+03  -4.603e+03  +1e+04  1e-02  5e-03  9e+00  2e+01  0.6298  1e-01   0  0  0 |  0  0
 3  +1.639e+03  -2.981e+02  +4e+03  3e-03  2e-03  4e+00  6e+00  0.8822  2e-01   0  0  0 |  0  0
 4  +1.630e+03  +9.231e+02  +1e+03  1e-03  6e-04  1e+00  2e+00  0.6739  8e-02   0  0  0 |  0  0
 5  +1.626e+03  +1.364e+03  +6e+02  4e-04  2e-04  5e-01  9e-01  0.6477  7e-02   1  0  1 |  0  0
 6  +1.623e+03  +1.560e+03  +2e+02  1e-04  7e-05  1e-01  2e-01  0.9890  3e-01   1  0  1 |  0  0
 7  +1.614e+03  +1.551e+03  +1e+02  1e-04  6e-05  1e-01  2e-01  0.2259  8e-01   1  1  1 |  0  0
 8  +1.611e+03  +1.571e+03  +9e+01  6e-05  4e-

1604.75223401161

In [31]:
np.sum(x.value)

9.798971105793418

Contrainte suivante

In [32]:
dej = np.zeros(n)
dej[3:23] = 1.0

In [33]:
contrainte5 = dej*x >=1.0

In [34]:
prob4= cp.Problem(diet1 , constraints = [contrainte1, contrainte2a,contrainte2b, contrainte3a, contrainte3b, contrainte4a, contrainte4b, contrainte5])
prob4.solve(verbose=True,solver="ECOS")


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +1.832e+03  -8.800e+04  +1e+05  2e-01  7e-02  1e+00  2e+02    ---    ---    1  1  - |  -  - 
 1  +1.649e+03  -1.372e+04  +3e+04  3e-02  1e-02  2e+01  5e+01  0.8864  1e-01   1  1  1 |  0  0
 2  +1.651e+03  -4.821e+03  +1e+04  1e-02  6e-03  9e+00  2e+01  0.6348  1e-01   0  0  0 |  0  0
 3  +1.642e+03  -5.917e+02  +4e+03  4e-03  2e-03  4e+00  7e+00  0.8023  2e-01   0  0  0 |  0  0
 4  +1.637e+03  +8.524e+02  +2e+03  1e-03  6e-04  2e+00  2e+00  0.7060  8e-02   0  0  0 |  0  0
 5  +1.634e+03  +1.331e+03  +6e+02  6e-04  2e-04  7e-01  9e-01  0.7119  1e-01   1  0  0 |  0  0
 6  +1.633e+03  +1.509e+03  +2e+02  2e-04  8e-05  3e-01  3e-01  0.6770  9e-02   1  0  0 |  0  0
 7  +1.633e+03  +1.580e+03  +8e+01  1e-04  3e-05  2e-01  1e-01  0.6623  5e-02   1  1  1 |  0  0
 8  +1.635e+03  +1.590e+03  +6e+01  1e-04  2e-

1651.6421331297972

In [35]:
xoptNew = x.value

In [36]:
for j in range(n):
    if xoptNew[j]>0.3:
        print("{} \t ({} cal) \t : \t {}".format(foods.iloc[j,0],foods.iloc[j,1],xoptNew[j]))


Baked_Hot_Apple_Pie 2.7_oz_(77_g)                          	 (250 cal) 	 : 	 0.9999999996684426
Chocolate_Chip_Cookie 1_cookie_(33_g)                      	 (160 cal) 	 : 	 1.0043893797538237
Coffee_(Large) 16_fl_oz_cup                                	 (0 cal) 	 : 	 0.3690274816713538
Coffee_(Medium) 16_fl_oz_cup                               	 (0 cal) 	 : 	 0.3690274816713538
Coffee_(Small) 12_fl_oz_cup                                	 (0 cal) 	 : 	 0.3690274816713538
Dasani_Water 16.9_fl_oz                                    	 (0 cal) 	 : 	 0.3690274816713538
Egg_McMuffin 4.8_oz_(135_g)                                	 (290 cal) 	 : 	 0.6330076555099001
Fat_Free_Chocolate_Milk_Jug 1_carton_(236_ml)              	 (130 cal) 	 : 	 0.6634884734114962
Hamburger 3.5_oz_(100_g)                                   	 (250 cal) 	 : 	 1.00654879227808
Hotcakes 5.3_oz_(151_g)                                    	 (350 cal) 	 : 	 1.7882217466289692
Side_Salad 3.1_oz_(87_g)                          

# 3. D'autres problèmes

Beaucoup d'autres problèmes de ce genre existent. Ci-dessous vous trouverez un nouveau problème mais vous êtes également invités à inventer vos propres problèmes et chercher si ils sont solvables par CVXPY ! 

> **Question :** Quels sont les aliments à choisir pour obtenir *au moins* les nutriments recommendés en un minimum d'unités ? 

In [37]:
y  = cp.Variable(n) 


efficace = cp.Minimize( np.ones(n) *  y)

In [38]:
contrainte = A*y >= b

In [39]:
problemEff = cp.Problem(efficace , constraints = [contrainte, y>=0])
problemEff.solve(verbose=True,solver="ECOS")


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +5.441e+00  +2.937e+02  +1e+03  3e-01  3e+00  1e+00  4e+00    ---    ---    1  1  - |  -  - 
 1  +1.631e+01  +1.518e+02  +6e+02  2e-01  2e+00  3e+00  2e+00  0.6946  2e-01   1  1  1 |  0  0
 2  +1.260e+01  +1.164e+02  +5e+02  1e-01  1e+00  2e+00  2e+00  0.2432  6e-01   1  1  1 |  0  0
 3  +1.007e+01  +6.130e+01  +3e+02  5e-02  7e-01  1e+00  1e+00  0.4805  2e-01   1  1  1 |  0  0
 4  +7.483e+00  +2.498e+01  +2e+02  1e-02  3e-01  4e-01  5e-01  0.7136  3e-01   1  1  1 |  0  0
 5  +6.284e+00  +1.795e+01  +1e+02  7e-03  2e-01  3e-01  4e-01  0.4344  6e-01   1  1  1 |  0  0
 6  +5.663e+00  +1.067e+01  +7e+01  2e-03  8e-02  1e-01  2e-01  0.5264  1e-01   1  1  1 |  0  0
 7  +5.148e+00  +9.870e+00  +6e+01  2e-03  8e-02  8e-02  2e-01  0.4406  7e-01   1  1  1 |  0  0
 8  +4.206e+00  +5.930e+00  +2e+01  7e-04  3e-

3.2072772481674483

In [40]:
y.value

array([ 1.01308411e-11,  1.38653699e-11,  1.77436332e-11,  2.64327771e-11,
        2.01666546e-11,  2.37661363e-11,  1.73222444e-11,  1.72739313e-11,
        1.43496844e-11,  1.93688092e-11,  1.47448989e-11,  1.71543525e-11,
        1.85965890e-11,  2.23346881e-11,  9.42474984e-12,  3.19438264e-11,
        2.51104509e-11,  1.55245788e-11,  1.51211436e-11,  2.27302417e+00,
        2.48285695e-10,  4.70011255e-11,  4.53223766e-11,  3.12916714e-11,
        9.09708304e-12,  7.84412433e-12,  6.66153174e-12,  1.69001017e-11,
        2.00342723e-11,  1.19142601e-11,  1.00756505e-11,  2.25193128e-11,
        1.56088756e-11,  1.17022159e-11,  1.81523623e-11,  1.03634031e-11,
        1.24173086e-11,  2.27259417e-11,  7.95757723e-12,  9.24421194e-12,
        1.08765417e-11,  1.30584524e-11,  1.30068395e-11,  7.80087390e-12,
        2.28641760e-11,  3.75175226e-11,  7.63654203e-11,  1.69057262e-11,
        2.79288857e-12,  7.29893577e-12,  5.91183756e-12,  4.38989286e-12,
       -1.47506476e-11, -

In [41]:
for j in range(n):
    if y.value[j]>0.3:
        print("{} \t ({} cal) \t : \t {}".format(foods.iloc[j,0],foods.iloc[j,1],y.value[j]))


Big_Breakfast_with_Hotcakes_(Large_Size_Biscuit) 15.3_oz_(434_g) 	 (1150 cal) 	 : 	 2.273024165714703
McFlurry_with_M&M'S_Candies_(16_fl_oz_cup) 16.2_oz_(460_g) 	 (930 cal) 	 : 	 0.37883735982826483
Premium_Southwest_Salad_with_Grilled_Chicken 11.8_oz_(335_g) 	 (290 cal) 	 : 	 0.35271064428563503


> **Question :** Quels nutriments sont les plus dur à obtenir (on pourra regarder la valeur de la variable duale) ? 

In [42]:
for i in range(len(b)):
    print("{} \t for {}".format(A.dot(y.value)[i],b[i]))


1361.9856303910965 	 for 600
151.7047681326478 	 for 65
53.74265185722111 	 for 20
1360.0914430177845 	 for 300
5464.794252713973 	 for 2400
343.2323206211901 	 for 300
98.92880472092085 	 for 50
100.00000000302231 	 for 100
100.0000000047785 	 for 100
100.00000000542161 	 for 100
100.00000000069555 	 for 100


In [43]:
problemEff.constraints[0].dual_value

array([2.41411413e-14, 2.09469785e-13, 4.06955292e-13, 3.10180270e-14,
       5.42025090e-15, 3.13134343e-12, 2.30460188e-13, 3.24226155e-03,
       2.43902439e-03, 1.09171433e-02, 1.54743433e-02])

# 4. Programmation entière

Les solutions obtenues précédemment sont des solutions *continues*, alors que l'on commande généralement un nombre entier d'items. Les problèmes d'optimisation avec variables entières appellés (Mixed) Integer Programs, se résolvent avec des méthodes et des solveurs différents. CVXPY appellant différents solveurs nativement, cette opération est transparente ici. 

### Retour sur le problème 2b

Les différences pour la programmation en nombre entier:

* le type de la variable 

        x = cp.Variable(n,integer=True) 

* le solver (qui doit etre capable de faire des MIP), par ex. "ECOS_BB" see [https://www.cvxpy.org/tutorial/advanced/index.html#choosing-a-solver](https://www.cvxpy.org/tutorial/advanced/index.html#choosing-a-solver)

        prob2.solve(verbose=True,solver="ECOS_BB")

In [44]:
x = cp.Variable(n,integer=True) 

cal = np.array(foods["Cal"]) # calories

diet1 = cp.Minimize(cal*x)

In [45]:
# Constraints matrix
A = np.array(foods.drop(columns=["Food","Cal"])).T
b = np.array(nutr)[0,1:12]

In [46]:
contrainte1 = x>=0

In [47]:
contrainte2a = A*x <= 1.0*b 
contrainte2b = A*x >= 0.8*b

In [54]:
prob2= cp.Problem(diet1 , constraints = [contrainte1, contrainte2a,contrainte2b])
#prob2.solve(verbose=True,solver="ECOS_BB")

In [49]:
prob.status

'infeasible'

### Relaxing Constraints

Les problemes MIP (Mixed Integer Programs) sont en général beaucoup plus dur car trouver des points admissibles est difficile, relaxons les contraintes (violemment... ex: 0<=x<=2 et Ax>=b).

In [50]:
contrainte1a = x>=0
contrainte1b = x<=2
contrainte2b = A*x >= b

In [55]:
prob2= cp.Problem(diet1 , constraints = [contrainte1a, contrainte1b, contrainte2b])
#prob2.solve(verbose=True,solver="ECOS_BB")

In [52]:
x.value

array([-6.88482693e-11,  2.00000000e+00, -2.35054104e-10, -3.59336718e-10,
       -3.79989176e-10, -1.66365346e-10, -1.94783651e-10, -1.88539093e-10,
       -1.96932166e-10, -2.07835190e-10, -1.87779835e-10, -1.61535647e-10,
       -2.48036352e-10, -2.97875481e-10, -9.89586288e-10, -3.68331955e-10,
       -4.05742231e-10, -4.05341231e-10, -4.05903941e-10, -4.02548244e-10,
       -3.89741335e-10, -4.10859775e-10, -4.06919618e-10, -4.15901689e-10,
       -2.10060635e-09, -1.81274493e-09, -2.34880543e-09, -1.02008092e-10,
       -1.47834953e-09, -1.71938512e-09, -1.37161273e-09, -1.29305929e-09,
       -1.30345816e-09, -1.36828886e-09, -1.05785686e-10, -6.28083726e-10,
       -7.09420639e-10, -6.35271050e-10, -3.11918483e-10, -4.36919024e-10,
       -7.15201271e-10, -2.88163192e-10, -2.11452080e-10,  2.00000000e+00,
       -1.22950952e-09, -1.26525502e-09, -1.17555116e-09, -8.46158861e-10,
       -3.27115252e-10,  3.88111091e-10,  2.07353585e-10, -1.56617215e-10,
        1.00000000e+00,  

In [53]:
for j in range(n):
    if x.value[j]>0.5:
        print("{} \t ({} cal) \t : \t {}".format(foods.iloc[j,0],foods.iloc[j,1],x.value[j]))


Apple_Slices 1.2_oz_(34_g)                                 	 (15 cal) 	 : 	 1.9999999988336503
Chocolate_Chip_Cookie 1_cookie_(33_g)                      	 (160 cal) 	 : 	 1.999999997183577
Coffee_(Large) 16_fl_oz_cup                                	 (0 cal) 	 : 	 1.0000000000000002
Coffee_(Medium) 16_fl_oz_cup                               	 (0 cal) 	 : 	 1.0000000000000002
Coffee_(Small) 12_fl_oz_cup                                	 (0 cal) 	 : 	 1.0000000000000002
Coffee_Cream 0.4_fl_oz_(11_ml)                             	 (20 cal) 	 : 	 0.9999999991337636
Dasani_Water 16.9_fl_oz                                    	 (0 cal) 	 : 	 1.0000000000000002
Diet_Coke_(Child) 12_fl_oz_cup                             	 (0 cal) 	 : 	 1.0000000000000604
Diet_Coke_(Large) 30_fl_oz_cup                             	 (0 cal) 	 : 	 1.0000000000000997
Diet_Coke_(Medium) 21_fl_oz_cup                            	 (0 cal) 	 : 	 1.000000000000074
Diet_Coke_(Small) 16_fl_oz_cup                            